## CORPUS PARA ORTH

In [1]:
import nltk
import re
import pandas as pd
import numpy as  np

def get_corpus_from_csv(filename,delimeter):
    coments = pd.read_csv(filename,sep=delimeter)
    corpus = []
    for i in range(len(coments)):
        oraciones = []
        sentence = coments.orth.iloc[i]
        sentence = sentence.split(' ')
        for word in sentence:
            if word != "[SILENCE]":
                word = word.replace('.','')
                word = word.replace(',','')
                word = word.replace('?','')
            
                word = word.lower()
                oraciones.append(word)
        corpus.append(oraciones)
    return corpus
filename = "train.sentences.pronunciations.multi.translations.csv"
delimiter = ';'

def get_corpus_distinct_from_corpus(corpus):
    corpus_distint = []
    for sentence in corpus:
        for word in sentence:
            if word not in corpus_distint:
                corpus_distint.append(word)
    return corpus_distint

corpus = get_corpus_from_csv(filename,delimiter)
corpus_distinct = get_corpus_distinct_from_corpus(corpus)

In [2]:
file = open("corpus.txt","w")  
for sentence in corpus:
    for  word  in sentence:
        file.write(word+" ")
    file.write("\n")
file.close() 

In [3]:
import tf_glove
model = tf_glove.GloVeModel(embedding_size=len(corpus_distinct), context_size=4)

## Entrenar el modelo

In [4]:
model.fit_to_corpus(corpus)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [9]:
model.train(num_epochs=500)

In [11]:
model.embedding_for("john").reshape(1,len(corpus_distinct))

array([[-0.69354796, -0.3923709 , -0.3853247 ,  0.43878925, -0.02851599,
         0.34692553,  0.07892638,  0.30675682, -0.5500167 , -1.7159796 ,
         0.76510316,  0.04077822, -0.04356343,  0.7014111 ,  0.07800671,
         0.7413534 , -0.9021892 ,  0.7863653 , -0.20411558, -0.86255133,
         1.2280962 , -1.1469352 , -0.17653808, -1.2350314 ,  0.8374455 ,
        -0.13905925,  0.35496873, -0.14604959, -0.00860152,  0.39901522,
         0.4340194 ,  0.17246588, -0.1950115 ,  0.7058366 , -0.5919082 ,
         0.5011158 , -1.1745951 , -0.5128975 , -0.23313515,  0.08170503,
        -0.14134687,  1.3697032 , -1.3980184 , -0.25380903, -0.09738231,
         0.8913454 ,  0.50387895,  0.12423915, -0.50868213, -0.72772807,
        -1.2128891 , -0.19833255,  0.61300844, -0.32998884,  0.8190512 ,
        -0.4389441 , -1.2499316 ,  0.20564741, -0.11851406,  0.574084  ,
         0.27699828,  0.22930425, -0.9359445 , -0.15061548,  0.6110869 ,
        -0.18205136,  0.12553231,  0.15472114,  1.1

## IMPLEMENTACIÓN KERAS

In [8]:
from pickle import dump,load
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential,load_model
from keras.utils.np_utils import to_categorical
import numpy as np
import io

Using TensorFlow backend.


## Abriendo archivo de traslation.txt

In [18]:
link="corpus.txt"
with io.open(link,encoding='utf-8') as f:
  data=f.read().lower() #.replace('\n',' \n ')
print('Corpus lenght in characters:',len(data))

Corpus lenght in characters: 3970


## Creando un token

In [19]:
tokenizer=Tokenizer()
corpus=data.lower().split("\n")    
tokenizer.fit_on_texts(corpus)
total_words=len(tokenizer.word_index) + 1
print('Total words: ',total_words)

Total words:  116


## Generar Secuencias

In [20]:
input_sequences=[]
for line in corpus:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    input_sequences.append(n_gram_sequence)
print('Total Sequences:',len(input_sequences))

Total Sequences: 574


In [21]:
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
print('Max Sequence Length:',max_sequence_len)

Max Sequence Length: 8


## Dividiendo las secuencias en etiquetas X(entradas - inputs) y y (salidas )

In [22]:
sequences=np.array(input_sequences)
X,y=sequences[:,:-1],sequences[:,-1]
y=to_categorical(y,num_classes=total_words,dtype='int8')

## Definir el modelo LSTM

In [23]:
model=Sequential()
model.add(Embedding(total_words,10,input_length=max_sequence_len-1))
model.add(LSTM(50))
model.add(Dense(total_words,activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 10)             1160      
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 116)               5916      
Total params: 19,276
Trainable params: 19,276
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Entrenar modelo

In [25]:
model.fit(X,y,epochs=350,verbose=1)

Epoch 1/350
574/574 [==============================] - 2s 4ms/step - loss: 4.7440 - acc: 0.0453
Epoch 2/350
574/574 [==============================] - 0s 292us/step - loss: 4.6907 - acc: 0.0767
Epoch 3/350
574/574 [==============================] - 0s 271us/step - loss: 4.4388 - acc: 0.0819
Epoch 4/350
574/574 [==============================] - 0s 275us/step - loss: 4.2292 - acc: 0.0819
Epoch 5/350
574/574 [==============================] - 0s 286us/step - loss: 4.1945 - acc: 0.0819
Epoch 6/350
574/574 [==============================] - 0s 281us/step - loss: 4.1779 - acc: 0.0819
Epoch 7/350
574/574 [==============================] - 0s 275us/step - loss: 4.1677 - acc: 0.0819
Epoch 8/350
574/574 [==============================] - 0s 282us/step - loss: 4.1568 - acc: 0.0819
Epoch 9/350
574/574 [==============================] - 0s 272us/step - loss: 4.1451 - acc: 0.0819
Epoch 10/350
574/574 [==============================] - 0s 282us/step - loss: 4.1319 - acc: 0.0819
Epoch 11/350
574/574 

574/574 [==============================] - 0s 372us/step - loss: 2.1758 - acc: 0.4321
Epoch 83/350
574/574 [==============================] - ETA: 0s - loss: 2.1605 - acc: 0.430 - 0s 457us/step - loss: 2.1565 - acc: 0.4321
Epoch 84/350
574/574 [==============================] - 0s 444us/step - loss: 2.1313 - acc: 0.4338 0s - loss: 2.1131 - acc: 0.43
Epoch 85/350
574/574 [==============================] - 0s 481us/step - loss: 2.1087 - acc: 0.4425
Epoch 86/350
574/574 [==============================] - 0s 541us/step - loss: 2.0949 - acc: 0.4512
Epoch 87/350
574/574 [==============================] - 0s 745us/step - loss: 2.0746 - acc: 0.4547
Epoch 88/350
574/574 [==============================] - 1s 939us/step - loss: 2.0572 - acc: 0.4512 0s - loss: 2.0591 - acc: 0.450
Epoch 89/350
574/574 [==============================] - 0s 595us/step - loss: 2.0382 - acc: 0.4669
Epoch 90/350
574/574 [==============================] - 0s 406us/step - loss: 2.0163 - acc: 0.4669
Epoch 91/350
574/574 [=

574/574 [==============================] - 0s 318us/step - loss: 1.2230 - acc: 0.6777
Epoch 162/350
574/574 [==============================] - 0s 309us/step - loss: 1.2189 - acc: 0.6794
Epoch 163/350
574/574 [==============================] - 0s 372us/step - loss: 1.2140 - acc: 0.6899
Epoch 164/350
574/574 [==============================] - 0s 299us/step - loss: 1.2120 - acc: 0.6812
Epoch 165/350
574/574 [==============================] - 0s 306us/step - loss: 1.2021 - acc: 0.6882
Epoch 166/350
574/574 [==============================] - 0s 297us/step - loss: 1.1999 - acc: 0.6916
Epoch 167/350
574/574 [==============================] - 0s 321us/step - loss: 1.1958 - acc: 0.6899
Epoch 168/350
574/574 [==============================] - 0s 300us/step - loss: 1.1876 - acc: 0.6864
Epoch 169/350
574/574 [==============================] - 0s 314us/step - loss: 1.1826 - acc: 0.6934
Epoch 170/350
574/574 [==============================] - 0s 303us/step - loss: 1.1831 - acc: 0.6899
Epoch 171/350


574/574 [==============================] - 0s 418us/step - loss: 0.9561 - acc: 0.7056
Epoch 242/350
574/574 [==============================] - 0s 386us/step - loss: 0.9530 - acc: 0.7195
Epoch 243/350
574/574 [==============================] - 0s 373us/step - loss: 0.9496 - acc: 0.7195
Epoch 244/350
574/574 [==============================] - 0s 379us/step - loss: 0.9497 - acc: 0.7143
Epoch 245/350
574/574 [==============================] - 0s 384us/step - loss: 0.9459 - acc: 0.7178
Epoch 246/350
574/574 [==============================] - 0s 406us/step - loss: 0.9432 - acc: 0.7247
Epoch 247/350
574/574 [==============================] - 0s 362us/step - loss: 0.9405 - acc: 0.7213
Epoch 248/350
574/574 [==============================] - 0s 395us/step - loss: 0.9420 - acc: 0.7195
Epoch 249/350
574/574 [==============================] - 0s 460us/step - loss: 0.9398 - acc: 0.7195
Epoch 250/350
574/574 [==============================] - 0s 336us/step - loss: 0.9369 - acc: 0.7265
Epoch 251/350


574/574 [==============================] - 0s 474us/step - loss: 0.8674 - acc: 0.7230 0s - loss: 0.8701 - acc: 0.7
Epoch 321/350
574/574 [==============================] - 0s 515us/step - loss: 0.8674 - acc: 0.7213
Epoch 322/350
574/574 [==============================] - 0s 549us/step - loss: 0.8608 - acc: 0.7265
Epoch 323/350
574/574 [==============================] - 0s 460us/step - loss: 0.8622 - acc: 0.7195
Epoch 324/350
574/574 [==============================] - 0s 295us/step - loss: 0.8648 - acc: 0.7213
Epoch 325/350
574/574 [==============================] - 0s 288us/step - loss: 0.8637 - acc: 0.7160
Epoch 326/350
574/574 [==============================] - 0s 329us/step - loss: 0.8627 - acc: 0.7247
Epoch 327/350
574/574 [==============================] - 0s 309us/step - loss: 0.8643 - acc: 0.7247
Epoch 328/350
574/574 [==============================] - 0s 297us/step - loss: 0.8588 - acc: 0.7230
Epoch 329/350
574/574 [==============================] - 0s 305us/step - loss: 0.8608

## Guardando el modelo ya entrenado

In [29]:
model.save('model_5.h5')
dump(tokenizer,open('tokenizer_5.pkl','wb'))

## Generando texto Neural Language Model

In [30]:
model = load_model('model_5.h5')
tokenizer = load(open('tokenizer_5.pkl','rb'))

## Función recursiva que genera texto

In [31]:
def generate_text(seed_text,next_words,max_sequence_len, model):
  for j in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  return seed_text

## Generando texto

In [34]:
text=generate_text("john buy icecream",7,max_sequence_len,model)
print(text)

john buy icecream car future not buy house sell yesterday
